# Algoritmo de Alon-Matias-Szegedy
En esta libreta programaremos el algoritmo de Alon-Matias-Szegedy (AMS) para estimar momentos en un flujo de datos.

Dado un flujo de tamaño $n$ constante, se definen \\( K \\) variables \\( X_1, X_2, \ldots, X_K \\) usando posiciones del flujo elegidas de forma aleatoria y uniforme. Estas variables almacenan un elemento \\( X_k.elemento \\) y un valor entero \\( X_k.valor \\), el cual se inicializa con 1 y se incrementa en 1 cada vez que se encuentra una ocurrencia de \\( X_k.elemento \\).
 
De esta forma, es posible estimar el \\( i \\)-ésimo momento a partir de una variable \\( X_k \\) calculando

$$
n \cdot (X_k.valor^{i} - (X_k.valor - 1)^{i})
$$

In [0]:
from collections import Counter
import numpy as np
np.random.seed(2021) # para reproducibilidad

n_variables = 50 #número de muestras para estimar los momentos

Definimos la clase para el algoritmo AMS

In [0]:
class AMS:  
  def __init__(self, n_variables=10):
    self.n_variables = n_variables

  def estima_momento(self, i):
    return np.mean(self.n * (self.valores**i - (self.valores - 1)**i)) #fórmula (esto produce un vector de k estimaciones distintas y se calcula el promedio)

  def calcula_cuentas(self, x):
    self.n = x.shape[0] #calculando cuántos elementos tiene el flujo
    self.ind = np.random.randint(0, self.n - 1, size=self.n_variables) #posiciones de forma aleatoria
    self.elementos = x[self.ind] #toma los elementos de esas posiciones
    self.valores = np.zeros_like(self.elementos) #inicializamos los valores
    for i,ind in enumerate(self.ind): #recorremos todos los índices 
      for j in range(ind, self.n): #recorremos cada una de las posiciones seleccionadas hasta el fin del flujo
        if self.elementos[i] == x[j]: #si encuentra el elemento
          self.valores[i] += 1 #incrementa el valor a 1

El momento $i$-ésimo está definido por
$$
\sum_{e\in \mathbb{U}} (m_e)^i
$$

donde $m_e$ es el número de veces que ocurre el elemento $e$ en el flujo y $\mathbb{U}$ es el conjunto universal.

In [0]:
#Función para calcular el momento
momento = lambda m, i: np.sum(m**i) 

Generamos un flujo de números enteros aleatorios.

In [0]:
x_min = 0 
x_max = 10
n = 100 #tamaño del flujo
flujo = np.random.randint(x_min, x_max, size=n)
flujo

Out[4]: array([4, 5, 9, 0, 6, 5, 8, 6, 6, 6, 6, 1, 5, 7, 1, 1, 5, 2, 0, 3, 1, 0,
       2, 6, 4, 8, 5, 1, 6, 7, 5, 6, 9, 5, 6, 9, 2, 4, 3, 9, 2, 8, 5, 3,
       1, 1, 9, 2, 7, 5, 3, 7, 3, 9, 4, 9, 7, 9, 7, 7, 4, 3, 2, 9, 2, 5,
       2, 0, 4, 7, 3, 1, 9, 4, 2, 3, 6, 6, 6, 1, 5, 7, 1, 9, 5, 0, 0, 4,
       9, 7, 3, 4, 7, 0, 3, 9, 6, 9, 3, 9])

In [0]:
#Calculamos las ocurencias para cada elemento distinto
print("Conteo de ocurrencias:",Counter(flujo))
print("Valores:",Counter(flujo).values())
print("Lista:",list(Counter(flujo).values()))

Conteo de ocurrencias: Counter({9: 15, 6: 13, 5: 12, 7: 11, 3: 11, 1: 10, 4: 9, 2: 9, 0: 7, 8: 3})
Valores: dict_values([9, 12, 15, 7, 13, 3, 10, 11, 9, 11])
Lista: [9, 12, 15, 7, 13, 3, 10, 11, 9, 11]


In [0]:
#Almacenando las ocurrencias de cada elemento
frec = np.array(list(Counter(flujo).values()))
print("Frecuencias:", frec)

Frecuencias: [ 9 12 15  7 13  3 10 11  9 11]


Instaciamos nuestra clase, calculamos las cuentas del elemento correspondiente a cada variable y estimamos los momentos 1, 2 y 3.

In [0]:
em = AMS(n_variables) #instanciamos la clase con el núm de muestras
em.calcula_cuentas(flujo) #calculamos las frecuencias

for i in range(1, 4):
  print(u'Momento {0}: Exacto = {1} Estimación = {2}'.format(i, momento(frec, i), em.estima_momento(i))) #recordemos: n es constante

Momento 1: Exacto = 100 Estimación = 100.0
Momento 2: Exacto = 1100 Estimación = 1128.0
Momento 3: Exacto = 12790 Estimación = 13444.0


Cuando el tamaño del flujo no es constante, seleccionamos las posiciones de las variables de la siguiente manera:
+ Se toman las primeras $s$ posiciones del flujo como variables.
+ Se elige la posición $n>s$ con probabilidad $\frac{s}{n}$
  + Si es elegida, se selecciona de forma aleatoria y uniforme una de las $s$ variables y se reemplaza por la de la posición $n$
  + En caso contrario se mantienen las posiciones de las $s$ variables    

In [0]:
class AMSFlujo:
  def __init__(self, n_variables):
    self.n_variables = n_variables
    self.i = 0 #contador de cuántos datos han pasado en el flujo
    self.elementos = np.zeros(self.n_variables) #arreglo para los elementos
    self.valores = np.zeros(self.n_variables) #arreglo para los valores

  def estima_momento(self, k):
    if self.i >= self.n_variables: #evaluamos: si el num_muestras visto es >= al num_muestras indicado
      return np.mean(self.i * (self.valores**k - (self.valores - 1)**k))
    else:
      return np.mean(self.i * (self.valores[:self.i]**k - (self.valores[:self.i] - 1)**k)) #toma los valores hasta la posición i
    
  def __call__(self, x): #este "x" es un dato del flujo
    if self.i < self.n_variables:
      self.elementos[self.i] = x
      self.valores[self.i] = 0 #inicializamos
    else:
      #analizo si sustituyo algunas de las variables 
      prob = self.n_variables / (self.i + 1) #caĺculo de la probabilidad
      j = np.random.choice([0, 1], p=[1 - prob, prob]) #para obtener 0= 1-proba y para 1=prob
      if j: #si "j" es 1 (entonces vamos a sustituir)
        pos = np.random.randint(0, self.n_variables) #nueva posición
        self.elementos[pos] = x
        self.valores[pos] = 0 #inicializamos
      
      #iteramos los elementos
      for a,e in enumerate(self.elementos):
        if e == x:
          self.valores[a] += 1 #aumenta en 1

    self.i += 1 #incrementa el valor de n

Instanciamos la clase y vamos agregando cada dato del flujo, actualizando las cuentas y estimando los momentos 1, 2 y 3.

In [0]:
emf = AMSFlujo(n_variables) #instanciamos la clase

In [0]:
for i in range(n):
  emf(flujo[i]) #pasamos posición por posición del flujo
  frec = np.array(list(Counter(flujo[:i+1]).values())) #calculamos el valor de las frecuencias
  
  #calculamos los k momentos por posición
  print(u'Posición {0}'.format(i)) 
  for k in range(1, 4):
    print(u'\tMomento {0}: Exacto = {1} Estimación = {2}'.format(k, momento(frec, k), emf.estima_momento(k)))

Posición 0
	Momento 1: Exacto = 1 Estimación = 1.0
	Momento 2: Exacto = 1 Estimación = -1.0
	Momento 3: Exacto = 1 Estimación = 1.0
Posición 1
	Momento 1: Exacto = 2 Estimación = 2.0
	Momento 2: Exacto = 2 Estimación = -2.0
	Momento 3: Exacto = 2 Estimación = 2.0
Posición 2
	Momento 1: Exacto = 3 Estimación = 3.0
	Momento 2: Exacto = 3 Estimación = -3.0
	Momento 3: Exacto = 3 Estimación = 3.0
Posición 3
	Momento 1: Exacto = 4 Estimación = 4.0
	Momento 2: Exacto = 4 Estimación = -4.0
	Momento 3: Exacto = 4 Estimación = 4.0
Posición 4
	Momento 1: Exacto = 5 Estimación = 5.0
	Momento 2: Exacto = 5 Estimación = -5.0
	Momento 3: Exacto = 5 Estimación = 5.0
Posición 5
	Momento 1: Exacto = 6 Estimación = 6.0
	Momento 2: Exacto = 8 Estimación = -6.0
	Momento 3: Exacto = 12 Estimación = 6.0
Posición 6
	Momento 1: Exacto = 7 Estimación = 7.0
	Momento 2: Exacto = 9 Estimación = -7.0
	Momento 3: Exacto = 13 Estimación = 7.0
Posición 7
	Momento 1: Exacto = 8 Estimación = 8.0
	Momento 2: Exacto = 12